In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_data = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_data = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

class RBM(nn.Module):
    def __init__(self, visible_units, hidden_units):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(hidden_units, visible_units) * 0.01)
        self.h_bias = nn.Parameter(torch.zeros(hidden_units))
        self.v_bias = nn.Parameter(torch.zeros(visible_units))

    def sample_hidden(self, v):
        prob_h = torch.sigmoid(torch.matmul(v, self.W.t()) + self.h_bias)
        return torch.bernoulli(prob_h), prob_h

    def sample_visible(self, h):
        prob_v = torch.sigmoid(torch.matmul(h, self.W) + self.v_bias)
        return torch.bernoulli(prob_v), prob_v

    def contrastive_divergence(self, v, k=1):
        v_sample = v
        for _ in range(k):
            h_sample, _ = self.sample_hidden(v_sample)
            v_sample, _ = self.sample_visible(h_sample)
        return v_sample

    def forward(self, v):
        h_sample, _ = self.sample_hidden(v)
        return h_sample

rbm = RBM(visible_units=784, hidden_units=128)
optimizer = optim.SGD(rbm.parameters(), lr=0.1)

def train_rbm(epochs=10):
    for epoch in range(epochs):
        loss_epoch = 0
        for data, _ in train_loader:
            data = data.view(-1, 784)
            v_sample = rbm.contrastive_divergence(data)
            loss = torch.mean((data - v_sample) ** 2)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()
        print(f"Epoch {epoch+1}, Loss: {loss_epoch/len(train_loader):.4f}")

train_rbm()

def extract_features(data_loader):
    features, labels = [], []
    with torch.no_grad():
        for data, target in data_loader:
            data = data.view(-1, 784)
            h_sample, _ = rbm.sample_hidden(data)
            features.append(h_sample.numpy())
            labels.append(target.numpy())
    return np.vstack(features), np.hstack(labels)

X_train, y_train = extract_features(train_loader)
X_test, y_test = extract_features(test_loader)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy: {accuracy:.4f}")


Epoch 1, Loss: 0.4811
Epoch 2, Loss: 0.4811
Epoch 3, Loss: 0.4810
Epoch 4, Loss: 0.4810
Epoch 5, Loss: 0.4811
Epoch 6, Loss: 0.4811
Epoch 7, Loss: 0.4809
Epoch 8, Loss: 0.4810
Epoch 9, Loss: 0.4810
Epoch 10, Loss: 0.4810
(60000, 128) (60000,)
(10000, 128) (10000,)
Classification Accuracy: 0.1339


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class RBM(nn.Module):
    def __init__(self, visible_units, hidden_units):
        super(RBM, self).__init__()
        self.W = nn.Parameter(torch.randn(hidden_units, visible_units) * 0.01)
        self.h_bias = nn.Parameter(torch.zeros(hidden_units))
        self.v_bias = nn.Parameter(torch.zeros(visible_units))

    def sample_hidden(self, v):
        prob_h = torch.sigmoid(torch.matmul(v, self.W.t()) + self.h_bias)
        return torch.bernoulli(prob_h), prob_h

    def sample_visible(self, h):
        prob_v = torch.sigmoid(torch.matmul(h, self.W) + self.v_bias)
        return torch.bernoulli(prob_v), prob_v

    def contrastive_divergence(self, v, k=1):
        v_sample = v
        for _ in range(k):
            h_sample, _ = self.sample_hidden(v_sample)
            v_sample, _ = self.sample_visible(h_sample)
        return v_sample

class StackRBM:
    def __init__(self, size=5):
        self.stack_size = size
        self.stack = torch.zeros((1, size))
        self.rbm = RBM(visible_units=size, hidden_units=size)
        self.optimizer = optim.SGD(self.rbm.parameters(), lr=0.1)

    def push(self, value):
        self.stack = torch.roll(self.stack, shifts=1, dims=1)
        self.stack[0, 0] = value
        self.train_rbm()

    def pop(self):
        top_value = self.stack[0, 0].item()
        self.stack[0, 0] = 0
        self.stack = torch.roll(self.stack, shifts=-1, dims=1)
        self.train_rbm()
        return top_value

    def train_rbm(self):
        v_sample = self.rbm.contrastive_divergence(self.stack)
        loss = torch.mean((self.stack - v_sample) ** 2)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def display_stack(self):
        print("Stack State:", self.stack.numpy())

stack_rbm = StackRBM(size=5)
stack_rbm.push(1)
stack_rbm.push(2)
stack_rbm.push(3)
stack_rbm.display_stack()
print("Popped:", stack_rbm.pop())
stack_rbm.display_stack()


Stack State: [[3. 2. 1. 0. 0.]]
Popped: 3.0
Stack State: [[2. 1. 0. 0. 0.]]
